In [1]:
%matplotlib inline
from pylab import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from networkx.drawing.nx_agraph import write_dot, graphviz_layout
from pyomo.environ import ConcreteModel, SolverFactory, TerminationCondition, \
    value, Var, Constraint, Expression, Objective, TransformationFactory, units as pyunits
from pyomo.network import Arc, SequentialDecomposition
from idaes.core.util.model_statistics import degrees_of_freedom

from split_test2 import Separator1

from mixer_example import Mixer1

### TODO: CREATE INTRO

##### Import WaterTAP3 Package

In [2]:
import watertap as wt

##### Step 1: Look up unit process library. Returns a List.

##### Step 3: Set up IDAES flowsheet for watertap. This selects a property package for you (see water_props.py). The property package could be changed in the future for performing heat/energy and other balances.

In [3]:
m = wt.watertap_setup(dynamic = False)

##### Step 3: Add a water source and define inlet flow.

In [4]:
m = wt.case_study_trains.get_case_study(name = 'carlsbad', flow = 4.5833, m = m)

/Users/amiara/NAWI/WaterTap/IDAES-WaterTAP3_v2/ml_regression.py:149: RuntimeWarning: divide by zero encountered in power
  return a*np.power(x, b)
/Users/amiara/NAWI/WaterTap/IDAES-WaterTAP3_v2/ml_regression.py:149: RuntimeWarning: divide by zero encountered in power
  return a*np.power(x, b)
/Users/amiara/NAWI/WaterTap/IDAES-WaterTAP3_v2/ml_regression.py:149: RuntimeWarning: divide by zero encountered in power
  return a*np.power(x, b)
/Users/amiara/NAWI/WaterTap/IDAES-WaterTAP3_v2/ml_regression.py:149: RuntimeWarning: divide by zero encountered in power
  return a*np.power(x, b)
/Users/amiara/NAWI/WaterTap/IDAES-WaterTAP3_v2/ml_regression.py:149: RuntimeWarning: divide by zero encountered in power
  return a*np.power(x, b)
/Users/amiara/NAWI/WaterTap/IDAES-WaterTAP3_v2/ml_regression.py:149: RuntimeWarning: divide by zero encountered in power
  return a*np.power(x, b)
/Users/amiara/NAWI/WaterTap/IDAES-WaterTAP3_v2/ml_regression.py:149: RuntimeWarning: divide by zero encountered in pow

In [5]:
wt.display.show_train2(model_name=m)

In [7]:
##### Step 5:  Run model and display results. The resutls display will be put into easier format to read for user soon.

In [6]:
wt.run_water_tap(m)

Ipopt 3.12.13: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.13, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      755
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      133

Total number of variables............................:      283
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        1
                     variables with only upper bounds:        0
Tot

In [8]:
unit_process_name = [
    "swoi", 
    "coag_floc",
    "tri_media_filtration",
    "SAA",
    "SBA",
    "cf",
    "ro",
    "lime",
    "co2",
    "chlor",
    "ammonia",
    "TWS_24_hr",
    "muni"
]


cost_variables = [
    "total_up_cost",
    "fixed_cap_inv_unadjusted",
    "fixed_cap_inv",
    "total_fixed_op_cost",
    "total_cap_investment",
    "land_cost",
    "working_cap",
    "catalysts_chemicals",#
    "electricity",##
    "cap_replacement_parts", #
    "salaries",
    "benefits",
    "maintenance",
    "lab",
    "insurance_taxes",
    "base_employee_salary_cost",
    
    "cat_and_chem_cost",##
    "labor_and_other_fixed",#
    "consumer_price_index",#
    
    "electricity_cost", ##
    "other_var_cost"   ## 
]

In [9]:
import pandas as pd
import numpy

def get_results_table(model, unit_process_name):
    data = {} 

    for variable in cost_variables:
        cost_list = []
        for node in unit_process_name:
            # call m.fs.'node'.costing.'variable'() for each node and variable
            costing_attr = getattr(m.fs, node).costing
            cost_method = getattr(costing_attr, variable) #m.fs.swoi.costing.total_up_cost
            
            if type(cost_method) == int:
                cost_method = np.float64(cost_method)
                
            if type(cost_method) == np.float64:
                cost_value = cost_method
            else:
                cost_value = cost_method()
            # append ^ to a list
            cost_list.append(cost_value)
        # assign 'variable': List['node' values] in a data dictionary
        data[variable] = cost_list


    df = pd.DataFrame(data = data, index = unit_process_name, columns = cost_variables)
    print(df)
    return df
    

In [10]:
get_results_table(m,unit_process_name)

                      total_up_cost  fixed_cap_inv_unadjusted  fixed_cap_inv  \
swoi                      23.366026                 20.308784      20.980305   
coag_floc                 15.121620                 11.765745      11.765745   
tri_media_filtration      14.283696                 11.748013      12.751178   
SAA                        0.917000                  0.313791       0.373451   
SBA                        1.967776                  0.351818       0.418708   
cf                        13.380566                 11.044252      11.987322   
ro                      1006.659115                236.288203     281.213213   
lime                      59.544991                 52.909213      52.909213   
co2                        8.515704                  7.646860       7.771228   
chlor                     10.362694                  8.699168       9.441991   
ammonia                    0.870994                  0.666818       0.793598   
TWS_24_hr                 13.271536     

,total_up_cost,fixed_cap_inv_unadjusted,fixed_cap_inv,total_fixed_op_cost,total_cap_investment,land_cost,working_cap,catalysts_chemicals,electricity,cap_replacement_parts,...,benefits,maintenance,lab,insurance_taxes,base_employee_salary_cost,cat_and_chem_cost,labor_and_other_fixed,consumer_price_index,electricity_cost,other_var_cost
swoi,23.366026,20.308784,20.980305,1.305235,22.060791,0.031470,1.049015,1.045314,0.0,1.033066,...,0.489075,0.167842,0.062941,0.041961,0.020309,0.000000,1.032158,1.034582,0.0,-0.000000
coag_floc,15.121620,11.765745,11.765745,0.732484,12.371681,0.017649,0.588287,1.000000,0.0,1.000000,...,0.274514,0.094126,0.035297,0.023531,0.011766,1.008727,1.000000,1.000000,0.0,1.008727
tri_media_filtration,14.283696,11.748013,12.751178,0.875833,13.407863,0.019127,0.637559,1.094885,0.0,1.085390,...,0.336348,0.102009,0.038254,0.025502,0.011748,0.000000,1.144289,1.104141,0.0,-0.000000
SAA,0.917000,0.313791,0.373451,0.024180,0.392684,0.000560,0.018673,1.449793,0.0,1.190128,...,0.009154,0.002988,0.001120,0.000747,0.000314,0.250068,1.255215,1.260734,0.0,0.250068
SBA,1.967776,0.351818,0.418708,0.027110,0.440272,0.000628,0.020935,1.449793,0.0,1.190128,...,0.010263,0.003350,0.001256,0.000837,0.000352,0.750197,1.255215,1.260734,0.0,0.750197
cf,13.380566,11.044252,11.987322,0.775897,12.604669,0.017981,0.599366,1.094885,0.0,1.085390,...,0.293714,0.095899,0.035962,0.023975,0.011044,0.000000,1.144289,1.104141,0.0,-0.000000
ro,1006.659115,236.288203,281.213213,18.207686,295.695694,0.421820,14.060661,1.449793,0.0,1.190128,...,6.893012,2.249706,0.843640,0.562426,0.236288,0.000000,1.255215,1.260734,0.0,-0.000000
lime,59.544991,52.909213,52.909213,2.344555,55.634037,0.079364,2.645461,1.000000,0.0,1.000000,...,0.784769,0.423274,0.158728,0.105818,0.052909,0.783200,1.000000,1.000000,0.0,0.783200
co2,8.515704,7.646860,7.771228,0.344258,8.171446,0.011657,0.388561,1.022155,0.0,1.016264,...,0.115215,0.062170,0.023314,0.015542,0.007647,0.000000,1.015824,1.016997,0.0,-0.000000
chlor,10.362694,8.699168,9.441991,0.434440,9.928254,0.014163,0.472100,1.094885,0.0,1.085390,...,0.147645,0.075536,0.028326,0.018884,0.008699,0.000000,1.144289,1.104141,0.0,-0.000000


In [28]:
x = get_results_table(m,unit_process_name)
x.to_csv('cost_results.csv', index = True)

                      total_up_cost  fixed_cap_inv_unadjusted  fixed_cap_inv  \
swoi                      23.366026                 20.308783      20.980305   
coag_floc                 13.104165                 11.765745      11.765745   
tri_media_filtration      13.426630                 11.080272      12.026418   
SAA                        0.390702                  0.295020       0.351112   
SBA                        0.438049                  0.330773       0.393662   
cf                        12.580562                 10.416512      11.305979   
ro                       913.272581                213.839658     254.496571   
lime                      53.796621                 49.187757      49.187757   
co2                        7.925649                  7.130754       7.246728   
chlor                     10.021529                  8.429884       9.149713   
ammonia                    0.833376                  0.639317       0.760869   
TWS_24_hr                 12.463140     

In [35]:
####TO DO LOAD AND SAVE!!

In [36]:
#### SAVE TRAIN ####
# path = 'trains/Tutorial1_treatment_train_example.csv'
# wt.save_train(T, path)

In [37]:
# #### LOAD TRAIN ####
# path = 'trains/Tutorial1_treatment_train_example.csv'
# TT = wt.load_train(path)

In [38]:
# wt.display.show_train(TT)